## 1. Introduction
> With the increasing popularity of online payments, the risk of fraudulent transactions is also rising. This poses a significant financial threat to both businesses and consumers. The objective of this analysis is to identify the patterns between fraudulent and non-fraudulent payments to help customers to identify the legitimacy of their transactions. 

## 2. Import Needed Libraries and Dataset
> The dataset is collected from Kaggle, which contains historical information about fraudulent transactions which can be used to detect fraud in online payments. Link to source: https://www.kaggle.com/datasets/jainilcoder/online-payment-fraud-detection.
> To better understand the dataset, here's the basic information about the dataset:
> * Step: A unit where 1 step equals 1 hour
> * Type: Type of Online Transaction
> * Amount: Amount in Transaction
> * nameOrig: Name of the customer (more like codenames)
> * oldbalanceOrg: Balance of the sender before transaction
> * newbalanceOrig: Balance of the sender after transaction
> * nameDest: Name of the receipent (Destination)
> * oldbalanceDest: Balance of the receipent before transaction
> * newbalanceDest: Balance of the receipent after transaction
> * isFraud: Is it a fraud?
> * isFlaggedFraud: Did the system detect it was a fraud?

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [15]:
df = pd.read_csv("onlinefraud.csv")

## 3. Data Cleaning

In [16]:
# Checking the total numbers of empty values in each column
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

> * There are no missing values

In [17]:
# Getting the information about the dataset, to check the data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


> * Columns associated with money amount are type float64
> * Columns associated with name ar type object
> * Columns associated with boolean types ar int64 except step which represents count of hours

In [18]:
print("Amount of duplicated values in df: " , df.duplicated().sum())

Amount of duplicated values in df:  0


> * There are no duplicate values

> ### Data cleaning process is done:
> * There are no missing values.
> * There are no wrong data types.
> * There are no duplicate values.

## 4. Exploratory Data Analysis (EDA)

In [19]:
# Read the 1st 5rows to explore the data
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [20]:
# Read the shape (rows, cols) of dataset
df.shape

(6362620, 11)

In [7]:
# Describe the dataset
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


## 5. Data Mining / Data Modelling